In [1]:
# coding=utf-8
from typing import Dict
import time 
import pandas as pd 

import torch
from datasets import Dataset, load_dataset
from transformers import PreTrainedTokenizerFast, Seq2SeqTrainer, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments
from transformers.generation.configuration_utils import GenerationConfig

In [2]:
import sys, os
root = os.path.realpath('.').replace('\\','/').split('/')[0: -2]
root = '/'.join(root)
if root not in sys.path:
     sys.path.append(root)

from model.chat_model import TextToTextModel
from config import SFTconfig, InferConfig, T5ModelConfig
from utils.functions import get_T5_config

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [3]:
def get_dataset(file: str, split: str, encode_fn: callable, encode_args: dict,  cache_dir: str='.cache') -> Dataset:
    """
    Load a dataset
    """
    dataset = load_dataset('json', data_files=file,  split=split, cache_dir=cache_dir)

    def merge_prompt_and_responses(sample: dict) -> Dict[str, str]:
        # add an eos token note that end of sentence, using in generate.
        prompt = encode_fn(f"{sample['prompt']}[EOS]", **encode_args)
        response = encode_fn(f"{sample['response']}[EOS]", **encode_args)
        return {
            'input_ids': prompt.input_ids,
            'labels': response.input_ids,
        }

    dataset = dataset.map(merge_prompt_and_responses)
    return dataset

In [4]:
def sft_train(config: SFTconfig) -> None:

    # step 1. 加载tokenizer
    tokenizer = PreTrainedTokenizerFast.from_pretrained(config.tokenizer_dir)
    
    # step 2. 加载预训练模型
    model = None
    if os.path.isdir(config.finetune_from_ckp_file):
        # 传入文件夹则 from_pretrained
        model = TextToTextModel.from_pretrained(config.finetune_from_ckp_file)
    else:
        # load_state_dict
        t5_config = get_T5_config(T5ModelConfig(), vocab_size=len(tokenizer), decoder_start_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
        model = TextToTextModel(t5_config)
        model.load_state_dict(torch.load(config.finetune_from_ckp_file, map_location='cpu')) # set cpu for no exception
        
    # Step 4: Load the dataset
    encode_args = {
        'truncation': False,
        'padding': 'max_length',
    }

    dataset = get_dataset(file=config.sft_train_file, encode_fn=tokenizer.encode_plus, encode_args=encode_args, split="train")

    # Step 5: Define the training arguments
    # T5属于sequence to sequence模型，故要使用Seq2SeqTrainingArguments、DataCollatorForSeq2Seq、Seq2SeqTrainer
    # huggingface官网的sft工具适用于language model/LM模型
    generation_config = GenerationConfig()
    generation_config.remove_invalid_values = True
    generation_config.eos_token_id = tokenizer.eos_token_id
    generation_config.pad_token_id = tokenizer.pad_token_id
    generation_config.decoder_start_token_id = tokenizer.pad_token_id
    generation_config.max_new_tokens = 320
    generation_config.repetition_penalty = 1.5
    generation_config.num_beams = 1         # greedy search
    generation_config.do_sample = False     # greedy search

    training_args = Seq2SeqTrainingArguments(
        output_dir=config.output_dir,
        per_device_train_batch_size=config.batch_size,
        auto_find_batch_size=True,  # 防止OOM
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        learning_rate=config.learning_rate,
        logging_steps=config.logging_steps,
        num_train_epochs=config.num_train_epochs,
        optim="adafactor",
        report_to='tensorboard',
        log_level='info',
        save_steps=config.save_steps,
        save_total_limit=3,
        fp16=config.fp16,
        logging_first_step=config.logging_first_step,
        warmup_steps=config.warmup_steps,
        seed=config.seed,
        generation_config=generation_config,
    )

    # step 6: init a collator
    collator = DataCollatorForSeq2Seq(tokenizer, max_length=config.max_seq_len)
    
    # Step 7: Define the Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=collator,
    )

    # step 8: train
    trainer.train(
        # resume_from_checkpoint=True
    )

    loss_log = pd.DataFrame(trainer.state.log_history)
    log_dir = './logs'
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    loss_log.to_csv(f"{log_dir}/ie_task_finetune_log_{time.strftime('%Y%m%d-%H%M')}.csv")

    # Step 9: Save the model
    trainer.save_model(config.output_dir)

    return trainer
    

In [ ]:
config = SFTconfig()
config.finetune_from_ckp_file = InferConfig().model_dir
config.sft_train_file = './data/my_train.json'
config.output_dir = './model_save/ie_task'
config.max_seq_len = 512
config.batch_size = 16
config.gradient_accumulation_steps = 4
config.logging_steps = 20
config.learning_rate = 5e-5
config.num_train_epochs = 6
config.save_steps = 3000
config.warmup_steps = 1000
print(config)

In [ ]:
trainer = sft_train(config)

In [1]:
import sys, os
root = os.path.realpath('.').replace('\\','/').split('/')[0: -2]
root = '/'.join(root)
if root not in sys.path:
     sys.path.append(root)
import ujson, torch
from rich import progress

from model.infer import ChatBot
from config import InferConfig
from utils.functions import f1_p_r_compute
inf_conf = InferConfig()
inf_conf.model_dir = './model_save/ie_task/'
bot = ChatBot(infer_config=inf_conf)


In [2]:
ret = bot.chat('请抽取出给定句子中的所有三元组。给定句子：傅淑云，女，汉族，1915年出生，上海人')
print(ret)

[(傅淑云,民族,汉族),(傅淑云,出生地,上海),(傅淑云,出生日期,1915年)]


In [3]:
def text_to_spo_list(sentence: str) -> str:
    '''
    将输出转换为SPO列表，时间复杂度： O(n)
    '''
    spo_list = []
    sentence = sentence.replace('，',',').replace('（','(').replace('）', ')') # 符号标准化

    cur_txt, cur_spo, started = '',  [], False
    for i, char in enumerate(sentence):
        if char not in '[](),':
            cur_txt += char
        elif char == '(':
            started = True
            cur_txt, cur_spo = '' , []
        elif char == ',' and started and len(cur_txt) > 0 and len(cur_spo) < 3:
            cur_spo.append(cur_txt)
            cur_txt = ''
        elif char == ')' and started and len(cur_txt) > 0 and len(cur_spo) == 2:
            cur_spo.append(cur_txt)
            spo_list.append(tuple(cur_spo))
            cur_spo = []
            cur_txt = ''
            started = False
    return spo_list
print(text_to_spo_list(ret))

[('傅淑云', '民族', '汉族'), ('傅淑云', '出生地', '上海'), ('傅淑云', '出生日期', '1915年')]


In [4]:
test_data = []
with open('./data/test.json', 'r', encoding='utf-8') as f:
    test_data = ujson.load(f)

In [5]:
test_data[0:2]

[{'prompt': '请抽取出给定句子中的所有三元组。给定句子：查尔斯·阿兰基斯（charles aránguiz），1989年4月17日出生于智利圣地亚哥，智利职业足球运动员，司职中场，效力于德国足球甲级联赛勒沃库森足球俱乐部',
  'response': '[(查尔斯·阿兰基斯,出生地,圣地亚哥),(查尔斯·阿兰基斯,出生日期,1989年4月17日)]'},
 {'prompt': '请抽取出给定句子中的所有三元组。给定句子：《离开》是由张宇谱曲，演唱',
  'response': '[(离开,歌手,张宇),(离开,作曲,张宇)]'}]

In [6]:
prompt_buffer, batch_size, n = [], 32, len(test_data)
traget_spo_list, predict_spo_list = [], []
for i, item in progress.track(enumerate(test_data), total=n):
    prompt_buffer.append(item['prompt'])
    traget_spo_list.append(
        text_to_spo_list(item['response'])
    )

    if len(prompt_buffer) == batch_size or i == n - 1:
        torch.cuda.empty_cache()
        model_pred = bot.chat(prompt_buffer)
        model_pred = [text_to_spo_list(item) for item in model_pred]
        predict_spo_list.extend(model_pred)
        prompt_buffer = []

Output()

In [7]:
print(traget_spo_list[0:2], '\n\n\n',predict_spo_list[0:2])

[[('查尔斯·阿兰基斯', '出生地', '圣地亚哥'), ('查尔斯·阿兰基斯', '出生日期', '1989年4月17日')], [('离开', '歌手', '张宇'), ('离开', '作曲', '张宇')]] 


 [[('查尔斯·阿兰基斯', '国籍', '智利'), ('查尔斯·阿兰基斯', '出生地', '智利圣地亚哥'), ('查尔斯·阿兰基斯', '出生日期', '1989年4月17日')], [('离开', '歌手', '张宇'), ('离开', '作曲', '张宇')]]


In [8]:
print(len(predict_spo_list), len(traget_spo_list))

21636 21636


In [9]:
f1, p, r = f1_p_r_compute(predict_spo_list, traget_spo_list)
print(f"f1: {f1:.2f}, precision： {p:.2f}, recall: {r:.2f}")

f1: 0.74, precision： 0.75, recall: 0.73


In [2]:
# 测试一下对话能力
bot.chat(['你好', '请抽取出给定句子中的所有三元组。给定句子：江苏省赣榆海洋经济开发区位于赣榆区青口镇临海而建，2003年1月28日，经江苏省人民政府《关于同意设立赣榆海洋经济开发区的批复》（苏政复〔2003〕14号）文件批准为全省首家省级海洋经济开发区，','如何看待最近南方天气突然变冷？'])

['你好，有什么我可以帮你的吗？',
 '[(江苏省赣榆海洋经济开发区,成立日期,2003年1月28日)]',
 '南方地区气候干燥，气候寒冷，冬季寒冷，夏季炎热，冬季寒冷的原因很多，可能是由于全球气候变暖导致的。\n南方气候的变化可以引起天气的变化，例如气温下降、降雨增多、冷空气南下等。南方气候的变化可以促进气候的稳定，有利于经济发展和经济繁荣。\n此外，南方地区的气候也可能受到自然灾害的影响，例如台风、台风、暴雨等，这些自然灾害会对南方气候产生影响。\n总之，南方气候的变化是一个复杂的过程，需要综合考虑多方面因素，才能应对。']